---

In [ ]:
from pathlib import Path

path = Path().absolute()

In [ ]:
cd ../

---

## Load Dataset
---

In [ ]:
import pandas as pd

df = pd.read_csv(path / "assets/datasets/dataset-1.csv", delimiter=";")

## Instantiate Classifier
---

In [ ]:
from pipeline.classification import Classification

clf = Classification(n_jobs=4, target_offload="gpu:0")

## Analysis
---

In [ ]:
import pickle
from pipeline.classification import POS

X = list(df["texts"])
y = list(df["targets"])

# from sklearn.model_selection import train_test_split
#
# X, _1, y, _2 = train_test_split(
#     X,
#     y,
#     train_size=.1,
#     random_state=42,
#     stratify=y
# )

X_cleaned = clf.clean(X)
X_tokenized = clf.tokenize(X_cleaned)
# X_tokenized = clf.tokenize(X_cleaned, list(set(POS["tags"]) - set(["DET","INTJ","NUM","PRON","PROPN","PUNCT","X"])))
X_train, X_test, y_train, y_test = clf.train_test_split(X_tokenized, y)
best_hyperparameters, estimation, grid_search = clf.tuning(X_train, y_train)

with open(path / "assets/best_hyperparameters.pickle", "wb") as f:
    pickle.dump(best_hyperparameters, f)

# best_hyperparameters = {
#     'svc__C': 10000,
#     'svc__decision_function_shape': 'ovo',
#     'svc__gamma': 0.0001,
#     'svc__kernel': 'rbf',
#     'tfidfvectorizer__max_df': 0.2,
#     'tfidfvectorizer__min_df': 1,
#     'tfidfvectorizer__ngram_range': (1, 2),
#     'tfidfvectorizer__norm': 'l2'
# }

model = clf.train(X_train, y_train, best_hyperparameters)
y_pred = clf.test(model, X_test)
accuracy, mcc = clf.score(y_test, y_pred)

print("Accuracy:", accuracy)
print("MCC:", mcc)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
disp.plot()

plt.show()